In [93]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [98]:
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama

from langchain_core.messages import HumanMessage
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain
from dotenv import load_dotenv
from langchain_community.llms import LlamaCpp  
import ollama
from langchain_community.embeddings import OllamaEmbeddings

In [95]:
ollama_emb = OllamaEmbeddings(
    model="mxbai-embed-large",
)

In [99]:
llm = Ollama(model = 'llama3') 
chat_model = ChatOllama(model = 'llama3')

In [97]:
from langchain_core.messages import HumanMessage

chat_model.invoke([HumanMessage(content="Hi! I'm Bob")])



AIMessage(content="Hello Bob! Nice to meet you. Is there something on your mind that you'd like to chat about, or are you just looking for some friendly conversation?", response_metadata={'model': 'llama3', 'created_at': '2024-07-15T20:52:58.75611Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 134732968167, 'load_duration': 72577982459, 'prompt_eval_count': 15, 'prompt_eval_duration': 15167751000, 'eval_count': 33, 'eval_duration': 46883624000}, id='run-1b757a44-cfc2-4f1b-abe9-1b2bd3240536-0')

In [9]:
chat_model.invoke([HumanMessage(content="What is my name?")])

AIMessage(content="I'm happy to help! However, I don't have any information about your personal identity or name. I'm a large language model, I don't have the ability to know or remember individual names or personal details. Each time you interact with me, it's a new conversation and I don't retain any information from previous conversations. If you'd like to share your name with me, I'd be happy to learn it!", response_metadata={'model': 'llama3', 'created_at': '2024-07-11T22:12:18.581927Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 32133835583, 'load_duration': 8681875, 'prompt_eval_count': 10, 'prompt_eval_duration': 11879784000, 'eval_count': 87, 'eval_duration': 20240030000}, id='run-c02457ba-4507-4e5e-bb49-a2af3db5cbb3-0')

In [11]:
from langchain_core.messages import AIMessage

chat_model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='I know this one... Your name is Bob, right?', response_metadata={'model': 'llama3', 'created_at': '2024-07-11T22:13:17.756291Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 7962995000, 'load_duration': 11726958, 'prompt_eval_count': 35, 'prompt_eval_duration': 6513383000, 'eval_count': 13, 'eval_duration': 1420395000}, id='run-a7beb1ab-1e49-4b0f-8aa8-ee269f854bfb-0')

In [51]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory
)

from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(chat_model, get_session_history)



In [24]:
config = {"configurable": {"session_id": "abc2"}}

In [25]:
response = with_message_history.invoke(
    [HumanMessage(content = "Hi! I'm Bob")],
    config = config
)

response.content

"Nice to meet you, Bob! How's your day going so far?"

In [27]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config = config 
)

response.content

'Bob!'

In [29]:
from langchain_core.prompts import  ChatMessagePromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    
    [
        ("system",
          "You are a helpful assistant. Answer all questions to the best of your ability"
        
         ),
        MessagesPlaceholder(variable_name = "messages"),
        
    ]
)

chain = prompt|chat_model

In [30]:
response = chain.invoke({"messages": [HumanMessage(content="hi, I'm bob")]})

response.content

"Hi Bob! It's nice to meet you. I'm here to help with any questions or topics you'd like to discuss. What's on your mind today?"

In [31]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [32]:
config = {"configurable": {"session_id": "abc5"}}


In [33]:
response = with_message_history.invoke(
    
    [HumanMessage(content = "Hi! I'm Jim")],
    config = config
)

response.content

"Hi Jim! It's great to meet you. I'm here to help with any questions or topics you'd like to discuss. What's on your mind today?"

In [34]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'I told you earlier, Jim! Your name is Jim.'

In [35]:
from langchain_core.prompts import  ChatMessagePromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    
    [
        ("system",
          "You are a helpful assistant. Answer all questions to the best of your ability in {language}"
        
         ),
        MessagesPlaceholder(variable_name = "messages"),
        
    ]
)

chain = prompt|chat_model

In [47]:
response = chain.invoke({"messages": [HumanMessage(content ="hi! I am Bob")], "language": "Spanish"})

response.content

'¡Hola Bob! ¿En qué puedo ayudarte hoy? (Hello Bob! How can I help you today?)'

In [52]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [53]:
config = {"configurable": {"session_id": "abc11"}}

In [54]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "Spanish"},
    config=config,
)

response.content

'¡hola Todd! Soy tu asistente virtual y estoy aquí para ayudarte con cualquier pregunta o necesidad que tengas. ¿En qué puedo ayudarte hoy?'

In [55]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Spanish"},
    config=config,
)

response.content

'Todd, tu nombre es Todd!'

In [66]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=chat_model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content="hi! I'm bob"),
 AIMessage(content='hi!'),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [67]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | chat_model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

'You told me earlier that your name is Bob.'

In [70]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "whats 2 + 2"'

In [71]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

In [72]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

'I remember, your name is Bob!'

In [91]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="tell me a joke")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="")

Here's one:

Why don't scientists trust atoms?

Because they make up everything!

Hope that made you smile! Do you want another one?